## Taller introductorio sobre ROS

Introducción a los conceptos básicos de ROS.

**Dictante:** _Ing. Emiliano J. Borghi Orué_ (eborghiorue@frba.utn.edu.ar) 

![Banner](media/banner.png)


### roscore

`roscore` es lo primero que hay que ejecutar en ROS porque se va a encargar del control de _nodos_.

In [ ]:
%%bash --bg 
roscore &

### ¿Qué es un nodo?

Un nodo es un componente de ROS (programa) que puede enviar y/o recibir mensajes (_tópicos_). 
`rosnode list` es un comando que permite ver los nodos activos:

In [ ]:
%%bash
rosnode list

In [ ]:
%%bash
rosnode info rosout

### Executando nodos con `rosrun`

`rosrun` es un comando que permite correr aquellos nodos que se encuentren dentro de un _paquete_.
En este caso, ejecutaremos un simulador en 2D de un robot tortuga:

In [ ]:
%%bash --bg
rosrun turtlesim turtlesim_node &

> **NOTA:** Este comando necesita que el Máster se esté ejecutando para poder funcionar

In [ ]:
%%bash
rosnode list

### ¿Cómo se comunican los nodos? 
`rostopic` permite obtener información acerca de los mensajes que se están intercambiando entre los nodos. Técnicamente, se conocen como **tópicos**.

Para ver los subcomandos que `rostopic` provee, puedo usar el flag de ayuda `-h`:

In [ ]:
%%bash
rostopic -h

`rostopic list` retorna una lista con todos los tópicos activos (que están publicando y/o suscribiéndose a un nodo).

In [ ]:
%%bash
rostopic list -v

Los _nodos_, para poder establecer el _tópico_ de la comunicación, deben estar hablando el mismo "idioma". Esto quiere decir que cada tópico debe tener un tipo de **mensaje** determinado. Para poder saber qué tipo de _mensaje_ está siendo utilizado en cierto _tópico_ puedo usar `rostopic type`.

In [ ]:
%%bash
rostopic type /turtle1/cmd_vel

Si quiero conocer en detalle cómo está constituido un mensaje, ROS me concede del comando `rosmsg`:

In [ ]:
%%bash
rosmsg show geometry_msgs/Twist

Como todo _tópico_, se puede crear un nodo con el cual publicar mensajes, por lo cual se usará `rostopic pub`.

In [ ]:
%%bash --bg
rostopic pub --once /turtle1/cmd_vel geometry_msgs/Twist -- \
'[2.0, 0.0, 0.0]' '[0.0, 0.0, 1.8]'

El comando previo envía un único mensaje a la tortuga diciéndole que se mueva con velocidad lineal de 2 m/s y una velocidad angular de 1.8 rad/s.

Veamos el comando anterior en detalle.

* El comando permite publicar mensajes en un cierto tópico:  
    `rostopic pub`
* Esta opción hace que se envien mensajes cada 1 Hz (una vez por segundo). Sin este parámetro, la tortuga se frenaría luego de cierto tiempo:  
    `-r 1` 
* Lo siguiente es el nombre del tópico al cual se quiere enviar el mensaje:  
    `/turtle1/cmd_vel`
* El tipo de mensaje que se debe usar para poder enviar el mensaje:  
    `geometry_msgs/Twist`
* Esta opción no es muy usual pero sirve para decirle al compilador que ninguno de los siguientes argumentos es una opción, ya que en caso de quiere usar números negativos podrían confundirse con el guión.  
    `--`
* Como se vio anteriormente, un mensaje del tipo `geometry_msgs/Twist` tiene dos vectores con tres elementos `float64` cada uno: `linear` y `angular`.
    `'[2.0, 0.0, 0.0]' '[0.0, 0.0, 1.8]'`


<img src="media/publish-subscribe.png" />

## Servicios
----
Los _Servicios_ son otra manera de comunicación entre nodos. Son un método de comunicación bidireccional donde uno realiza un pedido (**request**), y el otro, responde (**response**).
<img src="media/services.png" />

### Usando `rosservice`
`rosservice` tiene muchos comandos disponibles:

In [ ]:
%%bash
rosservice -h

El comando `list` muestra que el nodo `turtlesim` provee nueve servicios: `reset`, `clear`, `spawn`, `kill`, `turtle1/set_pen`, `/turtle1/teleport_absolute`, `/turtle1/teleport_relative`, `turtlesim/get_loggers`, and `turtlesim/set_logger_level`. Además, existen dos servicios relacionados con el nodo `rosout`: `/rosout/get_loggers` y `/rosout/set_logger_level`.

In [ ]:
%%bash
rosservice list

Para ver más en detenimiento sobre un servicio, basta con usar el comando `rosservice type`:

In [ ]:
%%bash
rosservice type clear

Este servicio es del tipo `Empty`, esto quiere decir que no necesita argumentos para ser llamado (envía un campo vacío como **request** y recibe otro campo vacío como **response**). Procedamos a utilizarlo con `rosservice call`. 


In [ ]:
%%bash
rosservice call clear "{}"

Como resultado de esta llamada, se borró el fondo de `turtlesim_node`.

Si en cambio deseamos llamar a un servicio que posee argumentos, como es el caso de `spawn`:

In [ ]:
%%bash
rosservice type spawn | rossrv show

Este servicio nos permite generar una nueva tortuga en el lugar que nosotros especifiquemos como argumento. El campo _name_ es opcional, por lo que no se lo daremos.

In [ ]:
%%bash
rosservice call spawn 2 2 0.2 ""


## Usando `rosparam`

`rosparam` permite almacenar y manipular datos en ROS ([más información](http://wiki.ros.org/Parameter%20Server)). `rosparam` usa el lenguaje _YAML_ como sintaxis. Algunos ejemplos pueden ser: `1` es un entero, `1.0` es un flotante, `one` es un _string_, `true` es un _boolean_, `[1, 2, 3]` es una lista de enteros, y `{a: b, c: d}` es un diccionario. Los comandos que pueden usarse con `rosparam` se listan a continuación:

In [ ]:
%%bash
rosparam -h

Veamos qué parámetros existen actualmente:

In [ ]:
%%bash
rosparam list

Para modificar uno de estos parámetros usaremos `rosparam set`:

In [ ]:
%%bash
rosparam set background_r 150

Para que este cambio tome efecto debe llamarse a:

In [ ]:
%%bash
rosservice call clear

Puede llamarse directamente a `rosparam get /` para acceder al contenido de todos los parámetros.

In [ ]:
%%bash
rosparam get /

## Acciones
<img src="media/actions.png"/>
Las *acciones* son llamadas asincrónicas a servicios.
Cuando se llama a una acción, se hace una solicitud a una funcionalidad que provee otro nodo. Es muy similar a los servicios, salvo que, cuando se llama a un servicio, debe esperar a que se finalice. En cambio, esto no es necesario para las acciones.

A continuación se ejecutará un servidor (`turtle_shape`) que permite enviar comandos de velocidad a la tortuga para dibujar formas:

In [ ]:
%%bash
rosrun turtle_actionlib shape_server

Y cada vez que se encuentra activo, pueden observarse cinco tópicos con características distintivas:

In [ ]:
%%bash
rostopic list | grep turtle_shape

### Llamando a un action server
Lo primero que debe hacerse es verificar qué tipo de mensaje requiere el `goal`:

In [ ]:
%%bash
rosmsg show `rostopic type /turtle_shape/goal`

### Llamando a un action server
Es un tipo de mensaje complejo, pero observemos qué sucede cuando se envía un mensaje para que la tortuga dibuje una forma de cinco lados:

In [ ]:
%%bash
rostopic pub -1 /turtle_shape/goal turtle_actionlib/ShapeActionGoal  -- \
"{header: {seq: 0, stamp: {secs: 0, nsecs: 0}, frame_id: ''}, \
goal_id: {stamp: {secs: 0, nsecs: 0}, id:''}, goal: {edges: 5, radius: 2.0}}"

### Llamando a un action server
Durante el recorrido puede cancelarse la acción llamando al nodo `turtle_shape/cancel`:

In [ ]:
%%bash
rostopic pub /turtle_shape/cancel actionlib_msgs/GoalID -- \
"{stamp: {secs: 0, nsecs: 0}, id: ''}"

Para tener un mayor control sobre el progreso de la acción, pueden accederse a los tópicos del servidor.
<img src="media/action-interface.png" />

## ¿Cómo se trabaja con ROS en la PC? 
### [Espacio de trabajo](http://wiki.ros.org/catkin/Tutorials/create_a_workspace)

Todo proyecto que vayamos a realizar contenerá muchos _nodos_ de ROS que estarán distribuidos dentro de _paquetes_.
Estos _paquetes_ tienen que estar dispuestos en un directorio reconocido por ROS para poder ser utilizados, y suele ser un lugar común donde el usuario crea sus _paquetes_ propios.
Esta zona se conoce como **espacio de trabajo** y se crea muy fácilmente. Se crea una carpeta vacía, y una vez dentro, se ejecuta el comando `catkin_make`. [Catkin](http://wiki.ros.org/catkin/conceptual_overview) es una herramienta para simplificar la compilación de código en ROS.

In [ ]:
%%bash 
mkdir -p ~/catkin_ws/src
cd ~/catkin_ws/
catkin_make


El paso anterior generó un espacio de trabajo (`catkin_ws`) pero ROS no está enterado aún sobre su existencia debido a que la variable de entorno no tiene asignado su valor.

**Nota:** No es estrictamente necesario que el workspace se llame así. En el caso del proyecto `create_autonomy`, se llama `create_ws`.

In [ ]:
%%bash 
echo $ROS_PACKAGE_PATH

#### Antes de usar nuestro espacio de trabajo

Para "dar aviso" a ROS, solo basta con tipear el siguiente comando **en el directorio `/catkin_ws`** (_¡importante!_).

In [ ]:
%%bash 
source ~/catkin_ws/devel/setup.bash

Una vez creado el espacio de trabajo se pueden agregar cuantos paquetes deseemos.

##### Una alternativa automática

Para que la ejecución anterior se realice cada vez que se abre una nueva terminal, se añade la instrucción al `.bashrc` (que es el programa que se ejecuta al abrir una nueva terminal) del siguiente modo:

In [ ]:
%%bash
echo "source ~/catkin_ws/devel/setup.bash" >> ~/.bashrc
source ~/.bashrc

## [Paquetes](http://wiki.ros.org/ROS/Tutorials/CreatingPackage)

Los _paquetes_ deben ser creados dentro de la carpeta `src` del espacio de trabajo, en nuestro caso, `~/catkin_ws/src`.
Una vez allí, se necesita un solo comando con la siguiente forma: 

`catkin_create_pkg <package_name> [depend1] [depend2] [depend3]`

Donde `<package_name>` es el nombre con el que queremos nombrar al paquete, y los siguientes argumentos son dependencias (otros paquetes de ROS que vamos a utilizar), que para el caso del Turtlebot, serán tres:

- `rospy` (necesario para crear nodos de ROS en Python)
- `geometry_msgs`
- `std_msgs`

Entonces, el comando quedará de la siguiente manera:


In [ ]:
%%bash
cd ~/catkin_ws/src
catkin_create_pkg mi_paquete rospy std_msgs geometry_msgs

El comando anterior nos generó un paquete con la siguiente estructura: 

In [ ]:
%%bash 
ls -lh ~/catkin_ws/src/mi_paquete

### [Compilar un paquete](http://wiki.ros.org/ROS/Tutorials/BuildingPackages)

Para compilar un paquete, es necesario volver a ejecutar `catkin_make` en el directorio root del espacio de trabajo.

In [ ]:
%%bash 
cd ~/catkin_ws/
catkin_make

#### ¿En qué se diferencia un directorio normal de un paquete de ROS?

ROS reconoce un paquete por contener dos archivos:

- El archivo `CMakeLists.txt` contiene toda la información necesaria para la compilación del paquete y está basado en el estándar utilizado para [cmake](https://cmake.org/overview/).
- El archivo `package.xml` contiene la metadata del paquete, esto es, dependencias, autor, versión, etc.

## iRobot Create 2 en Gazebo

Lo primero que hay que hacer es ejecutar el simulador _Gazebo_ con la _iRobot Create 2_.
Además, será necesario instalar algunas dependencias.
Esta sección se recomienda ejecutarla en el docker de `create_autonomy` y dividir la pantalla de la terminal usando [Tmux](https://www.hamvocke.com/blog/a-quick-and-easy-guide-to-tmux/) o abrir una nueva sesión de Docker.

In [ ]:
%%bash --bg 
export LOCALIZATION=slam
export RVIZ=true
export RVIZ_CONFIG=navigation
roslaunch ca_gazebo create_maze.launch &

#### Observaciones iniciales

Veamos que tópicos ofrece la simulación:

In [ ]:
%%bash
rostopic list

#### Mover al robot con el teclado

A continuación se muestra el código necesario para ejecutar un nodo que mueva el Create 2 usando el teclado de la computadora. Para que funcione, es **necesario abrirlo en una nueva terminal y mantener esta terminal seleccionada**: 

In [ ]:
roslaunch ca_tools keyboard_teleop.launch

#### Usar RViz para debuggear

RViz es un programa de ROS para visualizar datos de robots.

In [ ]:
%%bash --bg 
rviz -d `rospack find ca_tools`/rviz/robot_description.rviz

## Usando subscriptores

Para poder leer la información que publica el robot es necesario operar con subscriptores, para ello se provee un ejemplo de Python básico para escuchar un _tópico_ cualquiera. Modifíquelo para que pueda escucharse la **odometría**: 

In [ ]:
#!/usr/bin/env python
import rospy
from std_msgs.msg import String

def callback(data):
  # Función que se llama cada vez que el nodo recibe un dato
  rospy.loginfo("Valor leído: %s", data.data)

def main():

  # Se crea el nodo y se le asigna un nombre
  # SOLO PUEDE HABER UN SOLO NODO POR EJECUTABLE
  rospy.init_node('subscriber_node')
  # Hago que el nodo se suscriba a un String de nombre 'topic_name'
  #   y llame a callback() cada vez que recibe un dato nuevo
  rospy.Subscriber("topic_name", String, callback)
  # Como ROS no funciona en tiempo real, debe llamarse a spin() para procesar el callback()
  rospy.spin()

if __name__ == '__main__':
  main()

¿Observa algo usando `rosnode list`? ¿Qué sucede si publico en el tópico mencionado?

Los datos recibidos son estructuras, y en el caso del ejemplo (`std_msgs/String`) pueden conocerse usando un simple comando:

In [ ]:
%%bash 
rosmsg show std_msgs/String

Para dejar de ejecutar el node subscriptor, corra el siguiente comando:

In [ ]:
%%bash 
rosnode kill /subscriber_node

## Usando publicadores
El siguiente código de ejemplo muestra cómo enviar código a través de un _tópico_. Modifíquelo para poder mover el robot.
**Ayuda:** Para poder mover un robot del tipo diferencial usando mensajes `Twist`, se deben modificar los parámetros:
- Lineal > x
- Angular > z

A modo de recuerdo, la conformación del mensaje tipo `Twist` puede saberse usando:

In [ ]:
%%bash 
rosmsg show geometry_msgs/Twist

In [ ]:
#!/usr/bin/env python
import rospy
from std_msgs.msg import String

def main():
  pub = rospy.Publisher('topic_name', String, queue_size=10)
  rospy.init_node('publisher_node')
  # Variable que va a ser usada para generar un 'loop de tiempo'
  rate = rospy.Rate(10) # 10 Hz
  
 # Mientras no se cierre el nodo, ejecutar el código
  while not rospy.is_shutdown():
    texto = String()
    texto.data = "Hola!"
    pub.publish(texto)
    # Una vez procesado todo lo anterior, el código se va a pausar para enviar
    #   la información periódicamente según la variable 'rate'.
    # Tener en cuenta que, si este valor es muy bajo, el código no funcionará correctamente.
    rate.sleep()

if __name__ == '__main__':
  # Ejemplo básico de cómo atrapar excepciones en Python.
  # Cuando se mate el nodo publisher_node, no saltará una excepción.
  try:
    main()
  except rospy.ROSInterruptException:
    pass

Una manera sencilla para que el nodo `publisher_node` deje de publicar es matándolo por terminal o presionando `Ctrl + C`.

In [ ]:
%%bash 
rosnode kill /publisher_node

Ahora que sabemos como escribir y publicar en _tópicos_ por código en Python, ¿Cómo podemos poner todo junto en un **paquete**?.

### [Manejo de nodos remoto](http://wiki.ros.org/ROS/NetworkSetup#Setting_a_name_explicitly)

Para poder mover al Turtlebot desde otra computadora, será necesario configurar algunas opciones, pero antes de comenzar deberemos tener en cuenta que:
- En el robot correrá el **Máster**
- En otra computadora correrá el **Esclavo**

La configuración para el Máster es la siguiente:

`$ roscore`

`$ hostname -I`

`$ export ROS_IP=<IP_MASTER>`

Y para configurar la computadora remote se necesita:

`$ export ROS_MASTER_URI=http://IP_MASTER:11311`

`$ hostname -I`

`$ export ROS_IP=<IP_SLAVE>`

### [Niveles de verbosidad](http://wiki.ros.org/rospy/Overview/Logging)



In [ ]:
#!/usr/bin/env python
import rospy

if __name__ == '__main__':
  # Niveles de verbosidad: 
  # rospy.DEBUG
  # rospy.INFO
  # rospy.WARN
  # rospy.ERROR
  # rospy.FATAL
  rospy.init_node("log_verbosity_node", log_level=rospy.DEBUG)
  # Imprimiendo los diferentes tipos de mensajes
  rospy.logdebug("Debug")
  rospy.loginfo("Information")
  rospy.logwarn("Warning")
  rospy.logerr("Error")
  rospy.logfatal("Fatal")
